In [ ]:
%pip install fire
%pip install implicit

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
%cd /content/gdrive/My\ Drive/melon-playlist-continuation
!pwd

/content/gdrive/My Drive/melon-playlist-continuation
/content/gdrive/My Drive/melon-playlist-continuation


In [ ]:
run movielens.py


INFO:implicit:read data file in 9.507891654968262
DEBUG:implicit:training model bm25


DEBUG:implicit:trained model 'bm25' in 49.31867051124573
DEBUG:implicit:calculating top movies
DEBUG:implicit:calculating similar playlists
  0%|          | 0/153429 [00:00<?, ?it/s]

100%|██████████| 153429/153429 [00:00<00:00, 264547.15it/s]


In [ ]:
import pickle
with open('tag_id_map.pkl', 'rb') as f:
    my_dict = pickle.load(f)
    my_dict_inv = {} 
    for k, v in my_dict.items():
        my_dict_inv[v] = k    
f.close()


In [ ]:
import time
import pandas as pd
from arena_util import load_json
from arena_util import write_json
from arena_util import remove_seen
'''
train = pd.read_json('./arena_data/orig/train.json', encoding='UTF-8')
val = pd.read_json('./arena_data/questions/val.json', encoding='UTF-8')
song_meta = pd.read_json('./res/song_meta.json', encoding='UTF-8')
most_results = pd.read_json('./arena_data/results/results.json', encoding='UTF-8')
w2v_results = pd.read_json('./omg2.json', encoding='UTF-8')
'''
train = pd.read_json('./res/train.json', encoding='UTF-8')
val = pd.read_json('./res/val.json', encoding='UTF-8')
song_meta = pd.read_json('./res/song_meta.json', encoding='UTF-8')
most_results = pd.read_json('./omg2_real.json', encoding='UTF-8')
w2v_results = pd.read_json('./omg2_real.json', encoding='UTF-8')

song_dic = {} # plylst id - song id
tag_dic = {} # plylst id - tag
data = pd.concat([train, val])
data = data.set_index('id')

song_dic = data['songs'].to_dict()
tag_dic = data['tags'].to_dict()


"\ntrain = load_json('./arena_data/orig/train.json')\nval = load_json('./arena_data/questions/val.json')\nsong_meta = load_json('./res/song_meta.json')\nmost_results = load_json('./arena_data/results/results.json')\nsong_dic = {}\ntag_dic = {}\nfor t in train:\n    song_dic[t['id']] = t['songs']\n    tag_dic[t['id']] = t['tags']\nfor v in val:\n    song_dic[v['id']] = v['songs']\n    tag_dic[v['id']] = v['tags']\n"

In [ ]:
from tqdm import tqdm
import pickle
#f = open("./tmp1000.pkl", 'rb')
f = open("./tmp1000_real.pkl", 'rb')
x = pickle.load(f)
f.close()
#f = open("./tmp2000.pkl", 'rb')
f = open("./tmp2000_real.pkl", 'rb')
y = pickle.load(f)
f.close()

cnt = 0
answers1 = []
for index, q in tqdm(val.iterrows()):
    if q['songs'] != [] or q['tags'] != []:
        answers1.append({
            "id": q["id"],
            "songs": remove_seen(q["songs"], x[cnt])[:200],
            "tags": remove_seen(q["tags"], [my_dict_inv[_] for _ in y[cnt]])[:20],
        })
        cnt += 1
    
    else:
        answers1.append({
          "id": w2v_results.loc[index]["id"],
          "songs": w2v_results.loc[index]['songs'],
          "tags": w2v_results.loc[index]["tags"],
        }) 


23015it [00:10, 2265.70it/s]


"\ncnt = 0\n# check and update answer\nfor n, q in enumerate(answers):\n    if len(q['songs'])!=100:\n        cnt += 1\n        answers[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]\n    if len(q['tags'])!=10:\n        answers[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]\n"

In [ ]:
similar_playlist = {}
f = open("./similar-playlist.txt", 'r')
try:
  while True:
    a, b, _ = f.readline().split()
    a = int(a)
    b = int(b)
    if a==b:
      continue
    try:
      similar_playlist[a].append(b)
    except:
      similar_playlist[a] = [b]
except:
  print("end")
f.close()

end


In [ ]:
from tqdm import tqdm
import collections
answers2 = []
for index, q in tqdm(val.iterrows()):
    try:
        most_id = similar_playlist[q['id']]
        get_song = []
        get_tag = []
        for ID in most_id:
            get_song += song_dic[int(ID)]
            get_tag += tag_dic[int(ID)]
        get_song = list(pd.value_counts(get_song)[:200].index)
        get_tag = list(pd.value_counts(get_tag)[:20].index)
        
        answers2.append({
            "id": q["id"],
            "songs": remove_seen(q["songs"], get_song)[:200],
            "tags": remove_seen(q["tags"], get_tag)[:20],
        })
    
    except KeyError:
        answers2.append({
          "id": w2v_results.loc[index]["id"],
          "songs": w2v_results.loc[index]['songs'],
          "tags": w2v_results.loc[index]["tags"],
        }) 
    
cnt1 = 0
cnt2 = 0
'''
# check and update answer
for n, q in enumerate(answers2):
    if len(q['songs'])!=100:
        cnt1 += 1
        cnt2 += 100 - len(q['songs'])
        
        answers2[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]
    if len(q['tags'])!=10:
        answers2[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]


for n, q in enumerate(answers2):
    if len(q['songs'])>100:
        q['songs'] = q['songs'][:100]
    if len(q['tags'])>10:
        q['tags'] = q['tags'][:10]
    if len(q['songs'])!=100:
        answers2[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]
    if len(q['tags'])!=10:
        answers2[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]
  
'''

23015it [01:09, 329.34it/s]


"\n# check and update answer\nfor n, q in enumerate(answers2):\n    if len(q['songs'])!=100:\n        cnt1 += 1\n        cnt2 += 100 - len(q['songs'])\n        \n        answers2[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]\n    if len(q['tags'])!=10:\n        answers2[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]\n\n\nfor n, q in enumerate(answers2):\n    if len(q['songs'])>100:\n        q['songs'] = q['songs'][:100]\n    if len(q['tags'])>10:\n        q['tags'] = q['tags'][:10]\n    if len(q['songs'])!=100:\n        answers2[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]\n    if len(q['tags'])!=10:\n        answers2[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]\n  \n"

In [ ]:
answers3=answers2

In [ ]:
song = load_json("./res/song_meta.json")
for i, a in enumerate(answers1):
    tmp = []
    year = int(val.iloc[i]['updt_date'][:4])
    for s in a['songs']:
        y = int(song[s]['issue_date'][:4])
        if y <= year:
            tmp.append(s)
    a['songs'] = tmp
for i, a in enumerate(answers3):
    tmp = []
    year = int(val.iloc[i]['updt_date'][:4])
    for s in a['songs']:
        y = int(song[s]['issue_date'][:4])
        if y <= year:
            tmp.append(s)
    a['songs'] = tmp
for i, a in enumerate(answers4):
    tmp = []
    year = int(val.iloc[i]['updt_date'][:4])
    for s in a['songs']:
        y = int(song[s]['issue_date'][:4])
        if y <= year:
            tmp.append(s)
    a['songs'] = tmp


In [ ]:

answers = []
for i in range(len(answers1)):
  tmp = {}
  tmp_song = {}
  tmp_tag = {}
  for n, j in enumerate(answers1[i]['songs']):
      try:
        tmp_song[j] += 1 / (n+20)
      except:
        tmp_song[j] = 1 / (n+20)
  '''
  for n, j in enumerate(answers2[i]['songs']):
      try:
        tmp_song[j] += 1 / (n+200)
      except:
        tmp_song[j] = 1 / (n+30) / 2
  '''
  for n, j in enumerate(answers3[i]['songs']):
      try:
        tmp_song[j] += 1 / (n+30)
      except:
        tmp_song[j] = 1/ (n+30)
  for n, j in enumerate(answers4[i]['songs']):
      try:
        tmp_song[j] += 1 / (n+100)
      except:
        tmp_song[j] = 1 / (n+100)
          
  sorted_keys = sorted(tmp_song.items(), reverse=True, key=lambda item: item[1])
  sorted_keys = [k for (k, v) in sorted_keys]

  tmp['id'] = answers1[i]['id']
  tmp['songs'] = sorted_keys[:100]


  for n, j in enumerate(answers1[i]['tags']):
      try:
        tmp_tag[j] += 1 / (n+10)
      except:
        tmp_tag[j] = 1 / (n+10)
  for n, j in enumerate(answers2[i]['tags']):
      try:
        tmp_tag[j] += 1 / (n+15)
      except:
        tmp_tag[j] = 1/ (n+15)
  sorted_keys = sorted(tmp_tag.items(), reverse=True, key=lambda item: item[1])
  sorted_keys = [k for (k, v) in sorted_keys]




  tmp['tags'] = sorted_keys[:10]
  answers.append(tmp)

for index, q in tqdm(val.iterrows()):
    answers[i] = {
        "id": q["id"],
        "songs": remove_seen(q["songs"], answers[i]['songs'])[:100],
        "tags": remove_seen(q["tags"], answers[i]['tags'])[:10],
    }
cnt = 0
for n, q in enumerate(answers):
    if len(q['songs'])>100:
        q['songs'] = q['songs'][:100]
    if len(q['tags'])>10:
        q['tags'] = q['tags'][:10]
    if len(q['songs'])!=100:
        cnt += 1
        answers[n]['songs'] += remove_seen(q['songs'], most_results.loc[n]['songs'])[:100-len(q['songs'])]
    if len(q['tags'])!=10:
        answers[n]['tags'] += remove_seen(q['tags'], most_results.loc[n]['tags'])[:10-len(q['tags'])]
print(cnt)


23015it [00:05, 4598.29it/s]


537


In [ ]:
write_json(answers, "./results/myans.json")

In [ ]:
run evaluate.py evaluate --gt_fname=arena_data/answers/val.json --rec_fname=arena_data/results/myans.json

Music nDCG: 0.288084
Tag nDCG: 0.414938
Score: 0.307112


In [ ]:
answers1 : myALS
answers2 : 
answers3 : bm25-K=2
answers4 : bm25-K=6

In [ ]:
import myals
import imp
imp.reload(myals)


<module 'myals' from '/content/gdrive/My Drive/melon-playlist-continuation/myals.py'>